In [ ]:
%matplotlib widget
from collections import defaultdict
import glob
import sys
sys.path.append('../')


from joblib import dump, load
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import pacman_classes
from utils import tokenizer, analyzer


from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [ ]:
train_pacman = pacman_classes.PACManTrain(cycles_to_analyze=[24, 25])

In [ ]:
train_pacman.read_training_data(parallel=False)

In [ ]:
print(train_pacman.proposal_data['Cycle25'].iloc[0]['hand_classification'])


In [ ]:
train_pacman.proposal_data['Cycle25']

In [ ]:
train_pacman.fit_model(train_pacman.proposal_data['Cycle25'])

In [ ]:
train_pacman.apply_model(train_pacman.proposal_data['Cycle24'], training=True)

In [ ]:
train_pacman.proposal_data['Cycle24']

In [ ]:
train_pacman.proposal_data['Cycle24']['cleaned_text'].iloc[0]

In [ ]:
pred = train_pacman.model.predict(train_pacman.proposal_data['Cycle24']['cleaned_text'])
pred_prob = train_pacman.model.predict_proba(train_pacman.proposal_data['Cycle24']['cleaned_text'])

In [ ]:
pred_prob[0]

In [ ]:
train_pacman.proposal_data['Cycle24']['encoded_pred_classification'] = pred

In [ ]:
train_pacman.proposal_data['Cycle24']['pred_classification'] = train_pacman.encoder.inverse_transform(pred)

In [ ]:
train_pacman.proposal_data['Cycle24']

In [ ]:
train_pacman.model_results

In [ ]:
print(classification_report(train_pacman.model_results['encoded_hand_classification'], 
                            train_pacman.model_results['encoded_model_classification']))

In [ ]:
train_pacman.save_model_results(fout='pacman_results_cycle24.txt')

In [ ]:
analyze_pacman = analyzer.PACManAnalyze()
analyze_pacman.encoder = train_pacman.encoder

In [ ]:
analyze_pacman.compute_accuracy_measurements(df=train_pacman.model_results)

In [ ]:
analyze_pacman.computed_accuracy.head()

In [ ]:
# analyze_pacman.plot_barh(df=analyze_pacman.computed_accuracy, fout='../../model_results')

In [ ]:
for i, row in analyze_pacman.computed_accuracy.iterrows():
    row /= row.sum()
    print(row)

In [ ]:
cumulative = analyze_pacman.computed_accuracy.apply(sum, axis=0)

In [ ]:
cumulative/cumulative.sum()

In [ ]:
train_pacman.save_model(fname='pacman_production_model.joblib')

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
scores = cross_val_score(
    train_pacman.model, train_pacman.proposal_data['Cycle24']['cleaned_text'], train_pacman.proposal_data['Cycle24']['encoded_hand_classification'], cv=5, scoring='f1_macro')

In [ ]:
print(scores)

In [ ]:
def combine_proposals(pman):
    df1 = pman.proposal_data['Cycle24']
    df2 = pman.proposal_data['Cycle25']
    df = df2.append(df1, sort=True)
    return df

In [ ]:
train_pacman.proposal_data['Cycle24'].columns

In [ ]:
train_pacman.proposal_data['Cycle25'].columns

In [ ]:
train_pacman.proposal_data['Cycle24'].head()

In [ ]:
total_dataset = combine_proposals(train_pacman)

In [ ]:
total_dataset.loc[:,['encoded_hand_classification']].info()

In [ ]:
total_dataset['hand_classification'].unique()

In [ ]:
total_dataset.info()

In [ ]:
solar_system_df = total_dataset[total_dataset['hand_classification'] == 'solar system']


In [ ]:
indices = [i for i in range(30, 60, 1)]

In [ ]:
indices

In [ ]:
def get_balanced_subset(df, proposal_counts):
    min_num_proposals = proposal_counts.min()
    data = {}
    for proposal_type in df['hand_classification'].unique():
        proposal_df = df[df['hand_classification'] == proposal_type]
        indices = np.random.randint(low=0, high=len(proposal_df), size=100)
        data[proposal_type] = proposal_df.iloc[indices]
    
    final_df = pd.DataFrame()
    for key in data.keys():
        final_df = final_df.append(data[key])
    return final_df

In [ ]:
balanced_df = get_balanced_subset(total_dataset, a)

In [ ]:
a = balanced_df['hand_classification'].value_counts()

In [ ]:
weights = a/a.sum()

In [ ]:
weights.values

In [ ]:
scorestotal = cross_val_score(
    train_pacman.model, 
    total_dataset['cleaned_text'], 
    total_dataset['encoded_hand_classification'], 
    cv=4, 
    scoring='f1_macro'
)

In [ ]:
scorestotal

In [ ]:
scoresbalanced = cross_val_score(
    train_pacman.model, 
    balanced_df['cleaned_text'], 
    balanced_df['encoded_hand_classification'], 
    cv=4, 
    scoring='f1_macro'
)

In [ ]:
scoresbalanced

In [ ]:
print(f"{scorestotal.mean():.0%} +/- {scorestotal.std():.1%}")

In [ ]:
print(f"{scoresbalanced.mean():.0%} +/- {scoresbalanced.std():.1%}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(total_dataset['cleaned_text'], total_dataset['encoded_hand_classification'], test_size=0.15, random_state=42)

In [ ]:
len(y_train)/ len(total_dataset['cleaned_text'])

In [ ]:
train_df = pd.DataFrame()
train_df['cleaned_text'] = X_train
train_df['encoded_hand_classification'] = y_train

In [ ]:
test_df = pd.DataFrame()
test_df['cleaned_text'] = X_test
test_df['encoded_hand_classification'] = y_test

In [ ]:
train_pacman.fit_model(train_df)

In [ ]:
test_df

In [ ]:
pred = train_pacman.model.predict(test_df['cleaned_text'])

In [ ]:
print(classification_report(test_df['encoded_hand_classification'], pred))

In [ ]:
len(pred)

In [ ]:
data_out = defaultdict(list)
for i, row in pman.proposal_data['cycle_24'].iterrows():
    data_out['fname'].append(row['fname'])
    data_out['encoded_pred_classification'].append(row['encoded_pred_classification'])
    data_out['pred_classification'].append(row['pred_classification'])
    data_out['hand_classification'].append(row['hand_classification'])
    data_out['encoded_hand_classification'].append(row['encoded_hand_classification'])
    for j, class_prob in enumerate(pred_prob[i]):
        data_out[f"{pman.encoder.classes_[j].replace(' ','_')}_prob"].append(class_prob)